In [1]:
import pandas as pd


file_path = 'CustomerInteractionData.csv'
data = pd.read_csv(file_path)


data.head()


,RecordID,CustomerInteractionRawText,AgentAssignedTopic,LocationID,CallDurationSeconds,AgentID,CustomerID
0,1890041,Cus not satisfied with service. want to port o...,Port Out,8.0,10.0,z88Nr,5Ms7y
1,1890042,Jadav called to check his porting out status. ...,Port Out,1.0,6.0,xGAVI,s8Zyb
2,1890043,Customer thraetened to cancel her service. Sai...,Port Out,9.0,15.0,T7Kdd,3IB8J
3,1890044,called to ask steps for porting out. She is le...,Port Out,5.0,15.0,jtifd,mtR3W
4,1890045,Customer dissatisfied with support. Connectivi...,Port Out,7.0,13.0,hxY3d,DbBmN


In [2]:

data['LocationID'].fillna(-1, inplace=True)


median_call_duration = data['CallDurationSeconds'].median()
data['CallDurationSeconds'].fillna(median_call_duration, inplace=True)


data['AgentID'].fillna('Unknown', inplace=True)
data['CustomerID'].fillna('Unknown', inplace=True)


missing_values_after = data.isnull().sum()
missing_values_after


C:\Users\vedan\AppData\Local\Temp\ipykernel_10188\3526559753.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['LocationID'].fillna(-1, inplace=True)
C:\Users\vedan\AppData\Local\Temp\ipykernel_10188\3526559753.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

RecordID                      0
CustomerInteractionRawText    0
AgentAssignedTopic            0
LocationID                    0
CallDurationSeconds           0
AgentID                       0
CustomerID                    0
dtype: int64

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset

c:\Users\vedan\OneDrive\Desktop\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sklearn.model_selection import train_test_split  
from datasets import Dataset, DatasetDict
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling


file_path = 'CustomerInteractionData.csv'
data = pd.read_csv(file_path)


nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
STOPWORDS = set(stopwords.words('english'))
REGEX = re.compile(r'[^a-zA-Z0-9\s]')

def normalize_text(text):
    text = text.lower()
    text = REGEX.sub('', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in STOPWORDS]
    return ' '.join(tokens)

data['CustomerInteractionRawText'] = data['CustomerInteractionRawText'].apply(normalize_text)


dataset = Dataset.from_pandas(data[['CustomerInteractionRawText']])


train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


train_dataset = Dataset.from_pandas(train_data[['CustomerInteractionRawText']])
val_dataset = Dataset.from_pandas(val_data[['CustomerInteractionRawText']])

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['CustomerInteractionRawText'], padding='max_length', truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

dataset_split = tokenized_train_dataset.train_test_split(test_size=0.1)  
dataset_dict = DatasetDict({
    'train': dataset_split['train'],
    'test': dataset_split['test']
})

model = GPT2LMHeadModel.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'], 
)


trainer.train()

Map: 100%|██████████| 21/21 [00:00<00:00, 1780.37 examples/s]
c:\Users\vedan\OneDrive\Desktop\venv\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                               
 33%|███▎      | 19/57 [04:35<06:16,  9.90s/it]

{'eval_loss': 7.106177806854248, 'eval_runtime': 10.5092, 'eval_samples_per_second': 0.856, 'eval_steps_per_second': 0.19, 'epoch': 1.0}


                                               
 67%|██████▋   | 38/57 [07:53<02:32,  8.04s/it]

{'eval_loss': 6.600682735443115, 'eval_runtime': 8.8853, 'eval_samples_per_second': 1.013, 'eval_steps_per_second': 0.225, 'epoch': 2.0}


                                               
100%|██████████| 57/57 [11:09<00:00, 11.74s/it]

{'eval_loss': 6.4972662925720215, 'eval_runtime': 8.9409, 'eval_samples_per_second': 1.007, 'eval_steps_per_second': 0.224, 'epoch': 3.0}
{'train_runtime': 669.0778, 'train_samples_per_second': 0.327, 'train_steps_per_second': 0.085, 'train_loss': 6.3596962376644735, 'epoch': 3.0}


TrainOutput(global_step=57, training_loss=6.3596962376644735, metrics={'train_runtime': 669.0778, 'train_samples_per_second': 0.327, 'train_steps_per_second': 0.085, 'total_flos': 57222955008000.0, 'train_loss': 6.3596962376644735, 'epoch': 3.0})

In [5]:
eval_results = trainer.evaluate(eval_dataset=tokenized_val_dataset)
print(eval_results)

100%|██████████| 3/3 [00:13<00:00,  4.35s/it]

{'eval_loss': 5.759483337402344, 'eval_runtime': 19.6742, 'eval_samples_per_second': 1.067, 'eval_steps_per_second': 0.152, 'epoch': 3.0}


In [8]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
while(True):
    customer_interaction = input("Enter customer interaction: ")
    if customer_interaction == "exit":
        break
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    tokenizer.pad_token_id = tokenizer.eos_token_id

    input_ids = tokenizer.encode(customer_interaction, return_tensors='pt')

    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,
        num_return_sequences=1,
        temperature=1,
        top_k=50,
        do_sample=True,  
        top_p=0.95  
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print("Generated Response: ", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response:  pizza.

The event will take place in downtown Washington from 6 p.m. to 7 p.m. in the Village Village Apartments. The event will include a wine tasting with local chefs and food enthusiasts. Tickets are $17 for adults and $25 for children under 5.


In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
while(True):
    
    customer_name = input("Enter the customer's name: ")
    customer_interaction_history = input("Enter the customer's interaction history: ")

    customer_data = {
        "name": customer_name,
        "interaction_history": customer_interaction_history
    }
    if customer_name == "exit":
        break

    personalized_input = f"{customer_data['name']} has the following history: {customer_data['interaction_history']}"

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    tokenizer.pad_token_id = tokenizer.eos_token_id

    input_ids = tokenizer.encode(personalized_input, return_tensors='pt')

    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,
        num_return_sequences=1,
        temperature=0.9,  
        top_k=50,
        top_p=0.92,  
        do_sample=True,  
        early_stopping=True  
    )
    personalized_response = tokenizer.decode(output[0], skip_special_tokens=True)

    print("Personalized Response: ", personalized_response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Personalized Response:  vedant has the following history: sadistic and violent, and has a particular history of being an animal lover.

If you find that you are a child or young person who has a history of animal cruelty, please contact us to discuss this with the Animal Welfare Department.

Calls to see the animal welfare department are now closed.

Please contact us on 0319 723 754

Email to: animalwelfare@australia.gov.au.
